<img align="right" src="images/dans-small.png"/>
<img align="right" src="images/tf-small.png"/>
<img align="right" src="images/etcbc.png"/>

You might want to consider the [start](search.ipynb) of this tutorial.

In [1]:
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa

In [2]:
VERSION = '2017'
DATABASE = '~/github/etcbc'
BHSA = f'bhsa/tf/{VERSION}'
PARA = f'parallels/tf/{VERSION}'
TF = Fabric(locations=[DATABASE], modules=[BHSA, PARA], silent=True )

In [3]:
api = TF.load('', silent=True)
api.makeAvailableIn(globals())
B = Bhsa(api, 'search', version=VERSION)

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="{CORPUS} feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 4.3.1</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>


This notebook online:
<a target="_blank" href="http://nbviewer.jupyter.org/github/etcbc/bhsa/blob/master/tutorial/search.ipynb">NBViewer</a>
<a target="_blank" href="https://github.com/etcbc/bhsa/blob/master/tutorial/search.ipynb">GitHub</a>


# Relationships

There are relationship between objects that are about their identities, the way they 
occupy space, and the way they are connected.

Are two objects the same, do they occupy the same slots, do they overlap, is one embedded in the other,
does one come before the other? Is there an edge from one to another?

Although these relationships are easy to define, and even easy to implement,
they may be very costly to use. 
When searching, most of them have to be computed very many times.

Some of them have been precomputed and stored in an index, e.g. the embedding relationships.
They can be used without penalty.

Other relations are not suitable for pre-computing: most inequality relations are of that kind.
It would require an enormous amount of storage to pre-compute for each node the set of nodes that
occupy different slots. This type of relation will not be used in narrowing down the search space,
which means that it may take more time to get the results.

We are going to test all of our relationships here.

Let us first see what relationships we have:

In [4]:
S.relationsLegend()

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

The top of the list are identity and spatial relationships.
We are going to dicuss them.

The bottom of the list are relationships defined by the edge features of your dataset.
We have discussed them in [advanced](searchAdvanced.ipynb).

# Identity and order

## = equal as node

The `=` means that both parts are the same node. Left and right are not two things with similar properties,
no, they are one and the same thing.

Useful if the thing you search for it part of two wildly different patterns.

In [5]:
query = '''
v1:verse
  sentence
    clause rela=Objc
      phrase
        word sp=verb gn=f nu=pl
v2:verse
  sentence
    c1:clause
    c2:clause
    c3:clause
    c1 < c2
    c2 < c3
v1 = v2
'''
results = B.search(query)
B.table(results, end=10)

48 results


n | verse | sentence | clause | phrase | word | verse | sentence | clause | clause | clause
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | הָעֹמְדֹ֖ות 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּֽדְעִים֙  | לֵאמֹֽר׃ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | הָעֹמְדֹ֖ות 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | לֵאמֹֽר׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הָעֹמְדֹ֖ות  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הָעֹמְדֹ֖ות  | לֵאמֹֽר׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Jeremia&chapter=44&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Jeremiah 44:15">Jeremiah 44:15</a> | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים  | מְקַטְּרֹ֤ות  | מְקַטְּרֹ֤ות  | Jeremiah 44:15 | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃  | וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם  | הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס  | לֵאמֹֽר׃ 

In [6]:
B.show(results, start=1, end=1, colorMap={3: 'magenta', 7: 'cyan'})


##### Passage 1


## # unequal as node

`n # m` if `n` and `m` are not the same node.

If you write a template, and you know that one node should come before another one,
consider using `<` or `>`, which will constrain the results better.

We have seen this in action in the search for gapped phrases.

## < and > canonical order

`n < m` if `n` comes before `m` in the
[canonical ordering](https://dans-labs.github.io/text-fabric/Api/General/#navigating-nodes)
of nodes.

We have seen them in action before.

# Space occupation

## == same slots

Two objects are extensionally equal if they occupy exactly the same slots.

Quite an expensive relation, as you will see: nearly 30 seconds for ca. 3600 results.

This query may take half a minute.

In [7]:
indent(reset=True)
query = '''
v:verse
    s:sentence
v == s
'''
results = B.search(query)
B.table(results, end=10)
B.show(results, start=1, end=1)
info('Done')

3601 results


n | verse | sentence
--- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">Genesis 1:1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:16">Genesis 1:16</a> | וַיַּ֣עַשׂ אֱלֹהִ֔ים אֶת־שְׁנֵ֥י הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים אֶת־הַמָּאֹ֤ור הַגָּדֹל֙ לְמֶמְשֶׁ֣לֶת הַיֹּ֔ום וְאֶת־הַמָּאֹ֤ור הַקָּטֹן֙ לְמֶמְשֶׁ֣לֶת הַלַּ֔יְלָה וְאֵ֖ת הַכֹּוכָבִֽים׃ 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:1">Genesis 2:1</a> | וַיְכֻלּ֛וּ הַשָּׁמַ֥יִם וְהָאָ֖רֶץ וְכָל־צְבָאָֽם׃ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:9">Genesis 2:9</a> | וַיַּצְמַ֞ח יְהוָ֤ה אֱלֹהִים֙ מִן־הָ֣אֲדָמָ֔ה כָּל־עֵ֛ץ נֶחְמָ֥ד לְמַרְאֶ֖ה וְטֹ֣וב לְמַאֲכָ֑ל וְעֵ֤ץ הַֽחַיִּים֙ בְּתֹ֣וךְ הַגָּ֔ן וְעֵ֕ץ הַדַּ֖עַת טֹ֥וב וָרָֽע׃ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:5">Genesis 3:5</a> | כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=3&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 3:23">Genesis 3:23</a> | וַֽיְשַׁלְּחֵ֛הוּ יְהוָ֥ה אֱלֹהִ֖ים מִגַּן־עֵ֑דֶן לַֽעֲבֹד֙ אֶת־הָ֣אֲדָמָ֔ה אֲשֶׁ֥ר לֻקַּ֖ח מִשָּֽׁם׃ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=4&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 4:24">Genesis 4:24</a> | כִּ֥י שִׁבְעָתַ֖יִם יֻקַּם־קָ֑יִן וְלֶ֖מֶךְ שִׁבְעִ֥ים וְשִׁבְעָֽה׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=5&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 5:23">Genesis 5:23</a> | וַיְהִ֖י כָּל־יְמֵ֣י חֲנֹ֑וךְ חָמֵ֤שׁ וְשִׁשִּׁים֙ שָׁנָ֔ה וּשְׁלֹ֥שׁ מֵאֹ֖ות שָׁנָֽה׃ 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=5&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:5">Genesis 6:5</a> | וַיַּ֣רְא יְהוָ֔ה כִּ֥י רַבָּ֛ה רָעַ֥ת הָאָדָ֖ם בָּאָ֑רֶץ וְכָל־יֵ֨צֶר֙ מַחְשְׁבֹ֣ת לִבֹּ֔ו רַ֥ק רַ֖ע כָּל־הַיֹּֽום׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=6&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 6:8">Genesis 6:8</a> | וְנֹ֕חַ מָ֥צָא חֵ֖ן בְּעֵינֵ֥י יְהוָֽה׃ פ 


##### Passage 1


    23s Done


## && overlap

Two objects overlap if and only if they share at least one slot.
This is quite costly to use in some cases.

We are going to find the sentences that overlap with two verses.

In [8]:
query = '''
chapter
  v1:verse
  v2:verse
  s:sentence

s && v1
s && v2

v1 < v2
'''

This query does not perform well. That's why we call it in a more cautious way,
using the generic Text-Fabric `S` - API.
That will be explained in the [rough](searchRough.ipynb) part of the tutorial.

We fetch only 10 results, and still it may take a minute.

In [9]:
indent(reset=True)
results = S.search(query, limit=10)
info('Done')

    54s Done


In [10]:
B.table(results, end=10)
B.show(results, condensed=False, start=1, end=3)

n | chapter | verse | verse | sentence
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | Genesis 1:17 | Genesis 1:18 | וַיִּתֵּ֥ן אֹתָ֛ם אֱלֹהִ֖ים בִּרְקִ֣יעַ הַשָּׁמָ֑יִם לְהָאִ֖יר עַל־הָאָֽרֶץ׃ וְלִמְשֹׁל֙ בַּיֹּ֣ום וּבַלַּ֔יְלָה וּֽלֲהַבְדִּ֔יל בֵּ֥ין הָאֹ֖ור וּבֵ֣ין הַחֹ֑שֶׁךְ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | Genesis 1:29 | Genesis 1:30 | הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֲשֶׁר־בֹּו֙ נֶ֣פֶשׁ חַיָּ֔ה אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2">Genesis 2</a> | Genesis 2:4 | Genesis 2:7 | בְּיֹ֗ום עֲשֹׂ֛ות יְהוָ֥ה אֱלֹהִ֖ים אֶ֥רֶץ וְשָׁמָֽיִם׃ וַיִּיצֶר֩ יְהוָ֨ה אֱלֹהִ֜ים אֶת־הָֽאָדָ֗ם עָפָר֙ מִן־הָ֣אֲדָמָ֔ה 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7">Genesis 7</a> | Genesis 7:2 | Genesis 7:3 | מִכֹּ֣ל׀ הַבְּהֵמָ֣ה הַטְּהֹורָ֗ה תִּֽקַּח־לְךָ֛ שִׁבְעָ֥ה שִׁבְעָ֖ה אִ֣ישׁ וְאִשְׁתֹּ֑ו וּמִן־הַבְּהֵמָ֡ה אֲ֠שֶׁר לֹ֣א טְהֹרָ֥ה הִ֛וא שְׁנַ֖יִם אִ֥ישׁ וְאִשְׁתֹּֽו׃ גַּ֣ם מֵעֹ֧וף הַשָּׁמַ֛יִם שִׁבְעָ֥ה שִׁבְעָ֖ה זָכָ֣ר וּנְקֵבָ֑ה לְחַיֹּ֥ות זֶ֖רַע עַל־פְּנֵ֥י כָל־הָאָֽרֶץ׃ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7">Genesis 7</a> | Genesis 7:8 | Genesis 7:9 | מִן־הַבְּהֵמָה֙ הַטְּהֹורָ֔ה וּמִן־הַ֨בְּהֵמָ֔ה אֲשֶׁ֥ר אֵינֶ֖נָּה טְהֹרָ֑ה וּמִ֨ן־הָעֹ֔וף וְכֹ֥ל אֲשֶׁר־רֹמֵ֖שׂ עַל־הָֽאֲדָמָֽה׃ שְׁנַ֨יִם שְׁנַ֜יִם בָּ֧אוּ אֶל־נֹ֛חַ אֶל־הַתֵּבָ֖ה זָכָ֣ר וּנְקֵבָ֑ה כַּֽאֲשֶׁ֛ר צִוָּ֥ה אֱלֹהִ֖ים אֶת־נֹֽחַ׃ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=7&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 7">Genesis 7</a> | Genesis 7:13 | Genesis 7:14 | בְּעֶ֨צֶם הַיֹּ֤ום הַזֶּה֙ בָּ֣א נֹ֔חַ וְשֵׁם־וְחָ֥ם וָיֶ֖פֶת בְּנֵי־נֹ֑חַ וְאֵ֣שֶׁת נֹ֗חַ וּשְׁלֹ֧שֶׁת נְשֵֽׁי־בָנָ֛יו אִתָּ֖ם אֶל־הַתֵּבָֽה׃ הֵ֜מָּה וְכָל־הַֽחַיָּ֣ה לְמִינָ֗הּ וְכָל־הַבְּהֵמָה֙ לְמִינָ֔הּ וְכָל־הָרֶ֛מֶשׂ הָרֹמֵ֥שׂ עַל־הָאָ֖רֶץ לְמִינֵ֑הוּ וְכָל־הָעֹ֣וף לְמִינֵ֔הוּ כֹּ֖ל צִפֹּ֥ור כָּל־כָּנָֽף׃ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=9&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 9">Genesis 9</a> | Genesis 9:9 | Genesis 9:10 | וַאֲנִ֕י הִנְנִ֥י מֵקִ֛ים אֶת־בְּרִיתִ֖י אִתְּכֶ֑ם וְאֶֽת־זַרְעֲכֶ֖ם אַֽחֲרֵיכֶֽם׃ וְאֵ֨ת כָּל־נֶ֤פֶשׁ הַֽחַיָּה֙ אֲשֶׁ֣ר אִתְּכֶ֔ם בָּעֹ֧וף בַּבְּהֵמָ֛ה וּֽבְכָל־חַיַּ֥ת הָאָ֖רֶץ אִתְּכֶ֑ם מִכֹּל֙ יֹצְאֵ֣י הַתֵּבָ֔ה לְכֹ֖ל חַיַּ֥ת הָאָֽרֶץ׃ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10">Genesis 10</a> | Genesis 10:11 | Genesis 10:12 | וַיִּ֨בֶן֙ אֶת־נִ֣ינְוֵ֔ה וְאֶת־רְחֹבֹ֥ת עִ֖יר וְאֶת־כָּֽלַח׃ וְֽאֶת־רֶ֔סֶן בֵּ֥ין נִֽינְוֵ֖ה וּבֵ֣ין כָּ֑לַח 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10">Genesis 10</a> | Genesis 10:13 | Genesis 10:14 | וּמִצְרַ֡יִם יָלַ֞ד אֶת־לוּדִ֧ים וְאֶת־עֲנָמִ֛ים וְאֶת־לְהָבִ֖ים וְאֶת־נַפְתֻּחִֽים׃ וְֽאֶת־פַּתְרֻסִ֞ים וְאֶת־כַּסְלֻחִ֗ים אֲשֶׁ֨ר יָצְא֥וּ מִשָּׁ֛ם פְּלִשְׁתִּ֖ים וְאֶת־כַּפְתֹּרִֽים׃ ס 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10">Genesis 10</a> | Genesis 10:15 | Genesis 10:16 | וּכְנַ֗עַן יָלַ֛ד אֶת־צִידֹ֥ן בְּכֹרֹ֖ו וְאֶת־חֵֽת׃ וְאֶת־הַיְבוּסִי֙ וְאֶת־הָ֣אֱמֹרִ֔י וְאֵ֖ת הַגִּרְגָּשִֽׁי׃ וְאֶת־הַֽחִוִּ֥י וְאֶת־הַֽעַרְקִ֖י וְאֶת־הַסִּינִֽי׃ וְאֶת־הָֽאַרְוָדִ֥י וְאֶת־הַצְּמָרִ֖י וְאֶת־הַֽחֲמָתִ֑י 


##### Result 1



##### Result 2



##### Result 3


The following query is much faster and obtains all results!

In [11]:
query = '''
sentence
  =: w1:word
  w2:word
  :=

v1:verse
  wv1:word

v2:verse
  wv2:word

w1 = wv1
w2 = wv2
v1 < v2
'''

In [12]:
indent(reset=True)
results = B.search(query)
info('Done')

892 results
  3.16s Done


In [13]:
B.show(results, end=1)


##### Passage 1


## ## not the same slots

True when the two objects in question do not occupy exactly the same set of slots.
This is a very loose relationship.

## || disjoint slots

True when the two objects in question do not share any slots.
This is a rather loose relationship.

This can be used for locating gaps: a textual object that lies inside a gap of another object.
See also [gaps](searchGaps.ipynb).

## [[ and ]] embedding

`n [[ m` if object `n` embeds `m`.

`n ]] m` if object `n` lies embedded in `n`.

These relations are used implicitly in templates when there is indentation:

```
s:sentence
  p:phrase
    w1:word gn=f
    w2:word gn=m
```

The template above implicitly states the following embeddings:

* `s ]] p`
* `p ]] w1`
* `p ]] w2`

We have seen these relations in action.

# Positioning

## << and >> before and after (slot-wise)

These relations test whether one object comes before or after an other,
in the sense that the slots
occupied by the one object lie completely 
before or after the slots occupied by the other object.

In [14]:
query = '''
verse
  sentence
    c1:clause
    p:phrase
    c2:clause
    c1 << p
    c2 >> p
'''
results = B.search(query)
B.table(results, end=10)
B.show(results, condensed=False, start=1, end=1, colorMap={2: 'lightyellow', 3: 'cyan', 4: 'magenta', 5: 'blue'})

30933 results


n | verse | sentence | clause | phrase | clause
--- | --- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | עֹ֤שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | פְּרִי֙  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | לְמִינֹ֔ו  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | מַזְרִ֣יעַ זֶ֔רַע  | עֹ֤שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | מַזְרִ֣יעַ זֶ֔רַע  | פְּרִי֙  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ  | מַזְרִ֣יעַ זֶ֔רַע  | לְמִינֹ֔ו  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ  | עֹ֥שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ  | פְּרִ֛י  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ  | עֹ֥שֶׂה  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ  | מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ  | פְּרִ֛י  | אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו 


##### Result 1


## =: same start slots
This relation holds when the left and right hand sides are nodes that have the same first slot.
It serves to enforce the the children of a parent are textually the first things inside that
parent. We have seen it in action before.

## := same end slots
This relation holds when the left and right hand sides are nodes that have the same last slot
It serves to enforce the the children of a parent are textually the last things inside that
parent. We have seen it in action before.

## :: same boundary slots
This relation holds when `=:` and `:=` both hold between the left and right hand sides.
It serves to look for parents with single children, or at least, where the parent is textually spanned by a single child.

Let us look for a phrase, whose start and end slots coincide with its containing clause.
But only if the phrase does not conincide with its parent clause.

In [15]:
query = '''
verse
    c:clause
        :: p:phrase
c ## p
'''
results = B.search(query)
B.table(results, start=1, end=10)
B.show(results, start=1, end=1)

54 results


n | verse | clause | phrase
--- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=10&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 10:21">Genesis 10:21</a> | גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃  | גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃ 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=24&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 24:24">Genesis 24:24</a> | בַּת־בְּתוּאֵ֖ל אָנֹ֑כִי בֶּן־מִלְכָּ֕ה  | בַּת־בְּתוּאֵ֖ל בֶּן־מִלְכָּ֕ה 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:16">Genesis 31:16</a> | לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ  | לָ֥נוּ וּלְבָנֵ֑ינוּ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=31&verse=53&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 31:53">Genesis 31:53</a> | אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ יִשְׁפְּט֣וּ בֵינֵ֔ינוּ אֱלֹהֵ֖י אֲבִיהֶ֑ם  | אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ אֱלֹהֵ֖י אֲבִיהֶ֑ם 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:1">Exodus 28:1</a> | לְכַהֲנֹו־לִ֑י אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃  | לְכַהֲנֹו־אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=28&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 28:14">Exodus 28:14</a> | מִגְבָּלֹ֛ת תַּעֲשֶׂ֥ה אֹתָ֖ם מַעֲשֵׂ֣ה עֲבֹ֑ת  | מִגְבָּלֹ֛ת מַעֲשֵׂ֣ה עֲבֹ֑ת 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=29&verse=18&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 29:18">Exodus 29:18</a> | עֹלָ֥ה ה֖וּא לַֽיהוָ֑ה רֵ֣יחַ נִיחֹ֔וחַ  | עֹלָ֥ה רֵ֣יחַ נִיחֹ֔וחַ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:4">Exodus 30:4</a> | עַ֚ל שְׁתֵּ֣י צַלְעֹתָ֔יו תַּעֲשֶׂ֖ה עַל־שְׁנֵ֣י צִדָּ֑יו  | עַ֚ל שְׁתֵּ֣י צַלְעֹתָ֔יו עַל־שְׁנֵ֣י צִדָּ֑יו 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=30&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 30:13">Exodus 30:13</a> | זֶ֣ה׀ יִתְּנ֗וּ כָּל־מַחֲצִ֥ית הַשֶּׁ֖קֶל בְּשֶׁ֣קֶל הַקֹּ֑דֶשׁ  | זֶ֣ה׀ מַחֲצִ֥ית הַשֶּׁ֖קֶל בְּשֶׁ֣קֶל הַקֹּ֑דֶשׁ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=36&verse=35&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 36:35">Exodus 36:35</a> | מַעֲשֵׂ֥ה חֹשֵׁ֛ב עָשָׂ֥ה אֹתָ֖הּ כְּרֻבִֽים׃  | מַעֲשֵׂ֥ה חֹשֵׁ֛ב כְּרֻבִֽים׃ 


##### Passage 1


Here you see an extra phrases in such clauses, lying embedded in the clause-spanning phrase.

A nice case of **Mind the gap!**.


## <: adjacent before 
This relation holds when the left hand sides ends in a slot that lies before the first slot of the right hand side.
It serves to enforce an ordering between siblings of a parent.

## :> adjacent after
This relation holds when the left hand sides starts in a slot that lies after the last slot of the right hand side.

As an example: are there clauses with multiple clause atoms without a gap between the two?

In [16]:
query = '''
verse
    clause
        clause_atom
        <: clause_atom
'''
results = B.search(query)
B.table(results, start=1, end=10)

0 results


Conclusion: there is always textual material between clause_atoms of the same clause.
If we lift the adjacency to sequentially before (`<<`) we do get results:

In [17]:
query = '''
verse
    clause
        clause_atom
        << clause_atom
'''
results = B.search(query)
B.table(results, start=1, end=10)
B.show(results, start=1, end=1)

2589 results


n | verse | clause | clause_atom | clause_atom
--- | --- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7">Genesis 1:7</a> | וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ וּבֵ֣ין הַמַּ֔יִם  | וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙  | וּבֵ֣ין הַמַּ֔יִם 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י  | תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב  | עֵ֣ץ פְּרִ֞י 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:11">Genesis 1:11</a> | עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו עַל־הָאָ֑רֶץ  | עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו  | עַל־הָאָ֑רֶץ 
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ  | וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב  | וְעֵ֧ץ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:12">Genesis 1:12</a> | עֹ֥שֶׂה פְּרִ֛י לְמִינֵ֑הוּ  | עֹ֥שֶׂה פְּרִ֛י  | לְמִינֵ֑הוּ 
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=21&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:21">Genesis 1:21</a> | וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ לְמִֽינֵהֶ֗ם  | וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ  | לְמִֽינֵהֶ֗ם 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=29&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:29">Genesis 1:29</a> | הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ וְאֶת־כָּל־הָעֵ֛ץ  | הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀  | וְאֶת־כָּל־הָעֵ֛ץ 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=30&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:30">Genesis 1:30</a> | וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה  | וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ  | אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה 
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:17">Genesis 2:17</a> | כִּ֗י בְּיֹ֛ום מֹ֥ות תָּמֽוּת׃  | כִּ֗י בְּיֹ֛ום  | מֹ֥ות תָּמֽוּת׃ 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=2&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 2:22">Genesis 2:22</a> | וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע לְאִשָּׁ֑ה  | וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע  | לְאִשָּׁ֑ה 


##### Passage 1


# Nearness

The relations with `:` in their name always have a requirement somewhere that a slot of the
left hand node equals a slot of the right hand node, or that the two are adjacent.

All these relationships can be relaxed by a **nearness number**.
If you put a number *k* inside the relationship symbols, those restrictions will be relaxed to
*the one slot and the other slot should have a mutual distance of at most k*.

## =k: same start within k slots

Here is an example.

First we look for clauses, with a phrase in it that starts at the
same slot as the clause.

In [18]:
results = B.search('''
chapter book=Genesis chapter=1
    clause
      =: phrase
''')

126 results


Now we add a bit of freedom, but not much: 0. Indeed, this is no extra
freedom, and it should give the same number of results.

In [19]:
results=B.search('''
chapter book=Genesis chapter=1
    clause
      =0: phrase
''')

126 results


Now we add real freedom: 1 and 2

In [20]:
results = B.search('''
chapter book=Genesis chapter=1
    clause
      =1: phrase
''')

236 results


In [21]:
results = B.search('''
chapter book=Genesis chapter=1
    clause
      =2: phrase
''')

315 results


Let us see some cases:

In [22]:
B.table(results, start=1, end=10, linked=2)
B.show(results, condensed=False, start=1, end=4, colorMap={2: 'yellow', 3: 'cyan'})

n | chapter | clause | phrase
--- | --- | --- | ---
1 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </a> | בְּרֵאשִׁ֖ית 
2 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1">בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </a> | בָּרָ֣א 
3 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ </a> | וְ
4 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ </a> | הָאָ֗רֶץ 
5 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום </a> | וְ
6 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום </a> | חֹ֖שֶׁךְ 
7 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום </a> | עַל־פְּנֵ֣י תְהֹ֑ום 
8 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </a> | וְ
9 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:2">וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃ </a> | ר֣וּחַ אֱלֹהִ֔ים 
10 | Genesis 1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3">וַיֹּ֥אמֶר אֱלֹהִ֖ים </a> | וַ


##### Result 1



##### Result 2



##### Result 3



##### Result 4


The first and second result show the same clause, with its first and second phrase respectively.

Note that we look for phrases that lie embedded in their clause.
So we do not get phrases of a preceding clause.

But if we want, we can get those as well.

In [23]:
results = B.search('''
chapter book=Genesis chapter=1
    c:clause
    p:phrase
    
    c =2: p
''')

485 results


We have more results now. Here is a closer look:

In [24]:
B.table(results, start=1, end=10)
B.show(results, condensed=False, start=12, end=14, colorMap={2: 'yellow', 3: 'cyan'})

n | chapter | clause | phrase
--- | --- | --- | ---
1 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בְּרֵאשִׁ֖ית 
2 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃  | בָּרָ֣א 
3 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ  | וְ
4 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ  | הָאָ֗רֶץ 
5 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום  | וְ
6 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום  | חֹ֖שֶׁךְ 
7 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום  | עַל־פְּנֵ֣י תְהֹ֑ום 
8 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃  | וְ
9 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃  | ר֣וּחַ אֱלֹהִ֔ים 
10 | <a target="_blank" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1">Genesis 1</a> | וַיֹּ֥אמֶר אֱלֹהִ֖ים  | וַ


##### Result 12



##### Result 13



##### Result 14


Here you see in results 13 and 14 a phrase of the previous clause.

## :k= same end within k slots

## :k: same start and end within k slots

# Next

You are comfortable in space now.

Ready to enter a whole new dimension?
[quantifiers](searchQuantifiers.ipynb)

---

[basic](search.ipynb)
[advanced](searchAdvanced.ipynb)
relations
[quantifiers](searchQuantifiers.ipynb)
[rough](searchRough.ipynb)
[gaps](searchGaps.ipynb)